# Inicjalizacja kontekstu użytkownika

Cel: izolacja zasobów dla każdego uczestnika. Tworzone są schematy per użytkownik w katalogu `training_catalog`: `<user_slug>_bronze`, `<user_slug>_silver`, `<user_slug>_gold`. Uruchom tę komórkę przed dalszymi krokami w innych notebookach.

In [ ]:
# === Konfiguracja katalogów i schematów ===
CATALOG = "training_catalog"

# Pobierz aktualnego użytkownika (dla środowiska produkcyjnego)
# raw_user = spark.sql("SELECT current_user()").first()[0]
raw_user = "trainer"  # Dla środowiska szkoleniowego

import re
user_slug = re.sub(
    r'[^a-z0-9]', 
    '_', 
    raw_user.lower()
)

print(f"User slug: {user_slug}")

# Schematy per użytkownik (izolacja zasobów)
BRONZE_SCHEMA = f"{user_slug}_bronze"
SILVER_SCHEMA = f"{user_slug}_silver"
GOLD_SCHEMA   = f"{user_slug}_gold"

# === Ścieżka do datasetów ===
# Ścieżka bazowa do datasetów KION (Databricks Volume)
DATASET_BASE_PATH = "/Volumes/training_catalog/default/kion_datasets"

# Alternatywnie, jeśli używasz DBFS:
# DATASET_BASE_PATH = "/dbfs/FileStore/kion_datasets"

# Lub lokalna ścieżka dla notebooków (jeśli dane są w tym samym repo):
# import os
# DATASET_BASE_PATH = os.path.abspath("./dataset")

print(f"Dataset base path: {DATASET_BASE_PATH}")

# === Tworzenie katalogów i schematów ===
spark.sql(f'USE CATALOG {CATALOG}')

for s in [BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA]:
    spark.sql(f'CREATE SCHEMA IF NOT EXISTS {CATALOG}.{s}')

spark.sql(f'USE SCHEMA {BRONZE_SCHEMA}')

# === Wyświetl informacje o katalogu ===
print("\n=== Informacje o katalogu ===")
display(
    spark.sql(f"DESCRIBE CATALOG EXTENDED {CATALOG}")
)

# === Podsumowanie konfiguracji ===
print("\n=== Podsumowanie konfiguracji użytkownika ===")
display(
    spark.createDataFrame(
        [
            (
                raw_user, 
                f'{CATALOG}.{BRONZE_SCHEMA}', 
                f'{CATALOG}.{SILVER_SCHEMA}', 
                f'{CATALOG}.{GOLD_SCHEMA}',
                DATASET_BASE_PATH
            )
        ],
        ['user', 'bronze_schema', 'silver_schema', 'gold_schema', 'dataset_path']
    )
)

print("\n✓ Inicjalizacja zakończona pomyślnie!")
print(f"✓ Użytkownik: {raw_user}")
print(f"✓ Katalog: {CATALOG}")
print(f"✓ Schematy: {BRONZE_SCHEMA}, {SILVER_SCHEMA}, {GOLD_SCHEMA}")
print(f"✓ Ścieżka do danych: {DATASET_BASE_PATH}")
